Notebook 5: F2 and F_st

1. F_st in Hail if there is time
2. Heat maps for this analysis were plotted using R - will be kept that way
    - once you edit and organize the R codes, link them here 

## Index
1. [Setting Default Output Paths](#1.-Set-Default-Output-Paths)
2. [F2 analysis](#2.-F2-analysis)
3. [FST](#3.-F_st)
    1. [FST with PLINK](#3a.-F_st-with-PLINK)

# General Overview 
The purpose of this notebook is to show two population genetics analyses (F2 and FST) to understand recent and deep history. 

**This script contains information on how to:**
- Read in a matrix table (mt) and filter using sample IDs that were obtained from another matrix table 
- Separate a field into multiple fields
- Filter using the call rate field 
- Extract doubletons and check if they are the reference or alternate allele
- Count how many times a sample or a sample pair appears in a field 
- Combine two dictionaries and add up the values for identical keys
- Format list as pandas table 
- Export a matrix table as PLINK2 BED, BIM and FAM files 
- Set up a pair-wise comparison
- Drop certain fields
- Download a tool such as PLINK using a link and shell command 
- Run shell commands, and set up a script & run it from inside a code block 
- Calculate F_st in PLINK (once there is progress on this, I will elaborate more) 
- Go through a log file and extract needed information
- Write out results onto the Cloud 

Author: Mary T. Yohannes

In [ ]:
import hail as hl

# import the read_qc function
# tmp: this is commented out as the function will continue to change
#from read_qc_function import read_qc

import pandas as pd

## Set Requester Pays Bucket
Running through these tutorials, users must specify which project is to be billed. To change which project is billed, set the `GCP_PROJECT_NAME` variable to your own project.

In [ ]:
# setting requester pays bucket to use throughout tutorial
GCP_PROJECT_NAME = "diverse-pop-seq-ref" # change this to your project name
hl.init(spark_conf={
    'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
    'spark.hadoop.fs.gs.requester.pays.buckets': 'hgdp_tgp,gcp-public-data--gnomad',
    'spark.hadoop.fs.gs.requester.pays.project.id': GCP_PROJECT_NAME
})

### tmp read_qc function 
to be removed once tutorials & function are complete and we can troubleshoot importing

In [ ]:
def read_qc(
        default: bool = False,
        post_qc:bool = False,
        sample_qc: bool = False,
        variant_qc: bool = False,
        duplicate: bool = False,
        outlier_removal: bool = False,
        ld_pruning: bool = False,
        rel_unrel: str = 'default') -> hl.MatrixTable:
    """
    Wrapper function to get HGDP+1kGP data as Matrix Table at different stages of QC/filtering.
    By default, returns pre QC MatrixTable with qc filters annotated but not filtered.

    :param bool default: if True will preQC version of the dataset
    :param bool post_qc: if True will return a post QC matrix table that has gone through:
        - sample QC
        - variant QC
        - duplicate removal
        - outlier removal
    :param bool sample_qc: if True will return a post sample QC matrix table
    :param bool variant_qc: if True will return a post variant QC matrix table
    :param bool duplicate: if True will return a matrix table with duplicate samples removed
    :param bool outlier_removal: if True will return a matrix table with PCA outliers and duplicate samples removed
    :param bool ld_pruning: if True will return a matrix table that has gone through:
        - sample QC
        - variant QC
        - duplicate removal
        - LD pruning
        - additional variant filtering
    :param bool rel_unrel: default will return same mt as ld pruned above
        if 'all' will return the same matrix table as if ld_pruning is True
        if 'related_pre_outlier' will return a matrix table with only related samples pre pca outlier removal
        if 'unrelated_pre_outlier' will return a matrix table with only unrelated samples pre pca outlier removal
        if 'related_post_outlier' will return a matrix table with only related samples post pca outlier removal
        if 'unrelated_post_outlier' wil return a matrix table with only unrelated samples post pca outlier removal
    """
    # Reading in all the tables and matrix tables needed to generate the pre_qc matrix table
    sample_meta = hl.import_table('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/gnomad_meta_v1.tsv')
    sample_qc_meta = hl.read_table('gs://hgdp_tgp/output/gnomad_v3.1_sample_qc_metadata_hgdp_tgp_subset.ht')
    dense_mt = hl.read_matrix_table(
        'gs://gcp-public-data--gnomad/release/3.1.2/mt/genomes/gnomad.genomes.v3.1.2.hgdp_1kg_subset_dense.mt')
    
    dense_mt = dense_mt.naive_coalesce(5000)


    # Takes a list of dicts and converts it to a struct format (works with nested structs too)
    def dict_to_struct(d):
        fields = {}
        for k, v in d.items():
            if isinstance(v, dict):
                v = dict_to_struct(v)
            fields[k] = v
        return hl.struct(**fields)

    # un-flattening a hail table with nested structure
    # dict to hold struct names as well as nested field names
    d = {}

    # Getting the row field names
    row = sample_meta.row_value

    # returns a dict with the struct names as keys and their inner field names as values
    for name in row:
        def recur(dict_ref, split_name):
            if len(split_name) == 1:
                dict_ref[split_name[0]] = row[name]
                return
            existing = dict_ref.get(split_name[0])
            if existing is not None:
                assert isinstance(existing, dict), existing
                recur(existing, split_name[1:])
            else:
                existing = {}
                dict_ref[split_name[0]] = existing
                recur(existing, split_name[1:])
        recur(d, name.split('.'))

    # using the dict created from flattened struct, creating new structs now un-flattened
    sample_meta = sample_meta.select(**dict_to_struct(d))
    sample_meta = sample_meta.key_by('s')

    # grabbing the columns needed from HGDP metadata
    new_meta = sample_meta.select(sample_meta.hgdp_tgp_meta, sample_meta.bergstrom)

    # creating a table with gnomAD sample metadata and HGDP metadata
    ht = sample_qc_meta.annotate(**new_meta[sample_qc_meta.s])

    # stripping 'v3.1::' from the names to match with the densified MT
    ht = ht.key_by(s=ht.s.replace("v3.1::", ""))

    # Using hl.annotate_cols() method to annotate the gnomAD variant QC metadata onto the matrix table
    mt = dense_mt.annotate_cols(**ht[dense_mt.s])
    

    print(f"sample_qc: {sample_qc}\nvariant_qc: {variant_qc}\nduplicate: {duplicate}" \
          f"\noutlier_removal: { outlier_removal}\nld_pruning: {ld_pruning}\nrel_unrel: {rel_unrel}")
    
    if default:
        print("Returning default preQC matrix table")
        # returns preQC dataset
        return mt
    
    if post_qc:
        print("Returning post sample and variant QC matrix table with duplicates and PCA outliers removed")
        sample_qc = True
        variant_qc = True
        duplicate = True
        outlier_removal = True
    
    if sample_qc:
        print("Running sample QC")
        # run data through sample QC
        # filtering samples to those who should pass gnomADs sample QC
        # this filters to only samples that passed gnomad sample QC hard filters
        mt = mt.filter_cols(~mt.sample_filters.hard_filtered)

        # annotating partially filtered dataset with variant metadata
        mt = mt.annotate_rows(**dense_mt[mt.locus, mt.alleles])

    if variant_qc:
        print("Running variant QC")
        # run data through variant QC
        # Subsetting the variants in the dataset to only PASS variants (those which passed gnomAD's variant QC)
        # PASS variants are variants which have an entry in the filters field.
        # This field contains an array which contains a bool if any variant qc filter was failed
        # This is the last step in the QC process
        mt = mt.filter_rows(hl.len(mt.filters) != 0, keep=False)

    if duplicate:
        print("Removing any duplicate samples")
        # Removing any duplicates in the dataset using hl.distinct_by_col() which removes
        # columns with a duplicate column key. It keeps one column for each unique key.
        # after updating to the new dense_mt, this step is no longer necessary to run
        mt = mt.distinct_by_col()

    if outlier_removal:
        print("Removing PCA outliers")
        # remove PCA outliers and duplicates
        # reading in the PCA outlier list
        # To read in the PCA outlier list, first need to read the file in as a list
        # using hl.hadoop_open here which allows one to read in files into hail from Google cloud storage
        pca_outlier_path = 'gs://hgdp-1kg/hgdp_tgp/pca_outliers_v2.txt'
        with hl.utils.hadoop_open(pca_outlier_path) as file:
            outliers = [line.rstrip('\n') for line in file]

        # Using hl.literal here to convert the list from a python object to a hail expression so that it can be used
        # to filter out samples
        outliers_list = hl.literal(outliers)

        # Using the list of PCA outliers, using the ~ operator which is a negation operator and obtains the compliment
        # In this case the compliment is samples which are not contained in the pca outlier list
        mt = mt.filter_cols(~outliers_list.contains(mt['s']))

    if ld_pruning:
        print("Returning ld pruned post variant and sample QC matrix table pre PCA outlier removal ")
        # read in dataset which has additional variant filtering and ld pruning run
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt')

    if rel_unrel == "default":
        # do nothing
        # created a default value because there are multiple options for rel/unrel datasets
        mt = mt

    elif rel_unrel == 'related_pre_outlier':
        print("Returning post sample and variant QC matrix table " \
              "pre PCA outlier removal with only related individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only related individuals
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/rel_updated.mt')

        
    elif rel_unrel == 'unrelated_pre_outlier':
        print("Returning post QC matrix table with only unrelated individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only unrelated individuals
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt')


    elif rel_unrel == 'related_post_outlier':
        print("Returning post sample and variant QC matrix table " \
              "pre PCA outlier removal with only related individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only related individuals
        #   - PCA outlier removal
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt')


    elif rel_unrel == 'unrelated_pst_outlier':
        print("Returning post sample and variant QC matrix table " \
              "pre PCA outlier removal with only related individuals")
        # data has gone through:
        #   - sample QC
        #   - variant QC
        #   - duplicate removal
        #   - LD pruning
        #   - pc_relate - filter to only unrelated individuals
        #   - PCA outlier removal
        mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt')
        
    # Calculating both variant and sample_qc metrics on the mt before returning
    # so the stats are up to date with the version being written out
    mt = hl.sample_qc(mt)
    mt = hl.variant_qc(mt)
    
    return mt

# 1. Set Default Output Paths
These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets. The read_qc() function is intended to take the place of needing to write out and read in datasets by the user. 

By default we have commented out all of the write steps of the tutorials, if you would like to write out your own datasets, uncomment those sections and replace the paths with your own. 

[Back to Index](#Index)

In [3]:
# intermediate file - beginning input file for F2 analysis 
f2_input_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/pre_running_varqc.mt'

# unrelated samples mt - for subsetting purposes 
unrelated_path = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt'

# final F2 output 
f2_final_path = 'gs://hgdp-1kg/hgdp_tgp/FST_F2/F2/doubleton_sample_pair_count_tbl.csv'

# annotation file for F2 heat map 
f2_annotation_path = 'gs://hgdp-1kg/hgdp_tgp/FST_F2/F2/sampleID_pop_reg.txt'

# beginning input file for F_st analysis 
fst_input_path = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/whole_dataset.mt'

# path for exporting the PLINK files 
# include file prefix at the end of the path - here the prefix is 'hgdp_tgp'
plink_files_path = 'gs://hgdp-1kg/hgdp_tgp/FST_F2/FST/PLINK/unrel/hgdp_tgp'

# final F_st output  
fst_final_path = 'gs://hgdp-1kg/hgdp_tgp/FST_F2/FST/PLINK/mean_fst_sum_UPDATED.txt'

# 2. F2 analysis

The intermediate file being imported here is the file generated prior to any variant filter and LD pruning. We are running F2 on unrelated samples only and using the version generated after removing outliers and rerunning PCA (nb4) for subsetting. After obtaining the desired samples, we run Hail's common variant statistics so we can separate out doubletons - variants that shows up twice and only twice in a data set and useful in detecting rare variance & understanding recent history. Once we have the doubletons filtered, we then remove variants with call rate less than 0.05 (no more than 5% missingness/low missingness).


[Back to Index](#Index)

In [ ]:
# read-in intermediate file 
mt_filt = read_qc(post_qc=True)

# filter to only unrelated samples - 3380 samples 
mt_unrel = hl.read_matrix_table(unrelated_path) 
unrel_samples = mt_unrel.s.collect() # collect sample IDs as a list 
unrel_samples = hl.literal(unrel_samples) # capture and broadcast the list as an expression 
mt_filt_unrel = mt_filt.filter_cols(unrel_samples.contains(mt_filt['s'])) # filter mt 
print('Num of samples after filtering (unrelated samples) = ' + str(mt_filt_unrel.count()[1])) # 3380 samples

# run common variant statistics (quality control metrics)  
mt_unrel_varqc = hl.variant_qc(mt_filt_unrel)

# separate the AC array into individual fields and extract the doubletons  
mt_unrel_interm = mt_unrel_varqc.annotate_rows(AC1 = mt_unrel_varqc.variant_qc.AC[0], AC2 = mt_unrel_varqc.variant_qc.AC[1])
mt_unrel_only2 = mt_unrel_interm.filter_rows((mt_unrel_interm.AC1 == 2) | (mt_unrel_interm.AC2 == 2))
print('Num of variants that are doubletons = ' + str(mt_unrel_only2.count()[0])) # 18018978 variants 

# write out an intermediate file and read it back in (saving took ~23 min) - MIGHT NOT NEED THIS LATER (DECIDE ONCE RUN WITH THE NEW DS)
#mt_unrel_only2.write('gs://hgdp-1kg/hgdp_tgp/FST_F2/F2/doubleton.mt', overwrite=False)
#mt_unrel_only2 = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/FST_F2/F2/doubleton.mt')

# remove variants with call rate < 0.05 (no more than 5% missingness/low missingness)  
mt_unrel_only2_filtered = mt_unrel_only2.filter_rows(mt_unrel_only2.variant_qc.call_rate > 0.05)
print('Num of variants > 0.05 = ' + str(mt_unrel_only2_filtered.count()[0])) # 17997741 variants 

The next step is to check which allele, reference (ref) or alternate (alt), is the the doubleton. If the first element of the array in the allele frequency field (AF[0]) is less than the second elelement (AF[1]), then the doubleton is 1st allele (ref). If the first element (AF[0]) is greater than the second elelement (AF[1]), then the doubleton is 2nd allele (alt).

In [ ]:
# check allele frequency (AF) to see if the doubleton is the ref or alt allele 

# AF[0] < AF[1] - doubleton is 1st allele (ref)
mt_doubl_ref = mt_unrel_only2_filtered.filter_rows((mt_unrel_only2_filtered.variant_qc.AF[0] < mt_unrel_only2_filtered.variant_qc.AF[1]))
print('Num of variants where the 1st allele (ref) is the doubleton = ' + str(mt_doubl_ref.count()[0])) # 3159 variants


# AF[0] > AF[1] - doubleton is 2nd allele (alt)
mt_doubl_alt = mt_unrel_only2_filtered.filter_rows((mt_unrel_only2_filtered.variant_qc.AF[0] > mt_unrel_only2_filtered.variant_qc.AF[1]))
print('Num of variants where the 2nd allele (alt) is the doubleton = ' + str(mt_doubl_alt.count()[0])) # 17994582 variants

# sanity check 
mt_doubl_ref.count()[0] + mt_doubl_alt.count()[0] = mt_unrel_only2_filtered.count()[0]
print(3159 + 17994582 == 17997741) # True 

Once we have figured out which allele is the doubleton and divided the doubleton matrix table accordingly, the next step is to find the samples that have doubletons, compile them in a set, and annotate that onto the mt as a new row field. This done for each mt separately and can be achieved by looking at the genotype call (GT) field. When the doubleton is 1st allele (ref), the genotype call would be 0|1 & 0|0. When the doubleton is 2nd allele (alt), the genotype call would be 0|1 & 1|1. We chose a set for the results instead of a list because a list isn't hashable and the next step wouldn't have run. After the annotation of the new row field in each mt, we then count how many times a sample or a sample pair appears within that field and store the results in a dictionary. Once we have the two dictionaries (one for the ref and one for the alt), we merge them into one and add up the values for identical keys.

If you want to do a sanity check at this point, you can add up the count of the two dictionaries and then subtract the number of keys that intersect between the two. The value that you get should be equal to the length of the combined dictionary.  

In [ ]:
# for each mt, find the samples that have doubletons, compile them in a set, and add as a new row field 

# doubleton is 1st allele (ref) - 0|1 & 0|0
# if there is one sample in the new column field then it's 0|0. If there are two samples, then it's 0|1
mt_ref_collected = mt_doubl_ref.annotate_rows(
    samples_with_doubletons = hl.agg.filter(
        (mt_doubl_ref.GT == hl.call(0, 1))| (mt_doubl_ref.GT == hl.call(0, 0)), hl.agg.collect_as_set(mt_doubl_ref.s)))

# doubleton is 2nd allele (alt) - 0|1 & 1|1
# if there is one sample in the new column field then it's 1|1. If there are two samples, then it's 0|1
mt_alt_collected = mt_doubl_alt.annotate_rows(
    samples_with_doubletons = hl.agg.filter(
        (mt_doubl_alt.GT == hl.call(0, 1))| (mt_doubl_alt.GT == hl.call(1, 1)), hl.agg.collect_as_set(mt_doubl_alt.s)))

# count how many times a sample or a sample pair appears in the "samples_with_doubletons" field - returns a dictionary
ref_doubl_count = mt_ref_collected.aggregate_rows(hl.agg.counter(mt_ref_collected.samples_with_doubletons))
alt_doubl_count = mt_alt_collected.aggregate_rows(hl.agg.counter(mt_alt_collected.samples_with_doubletons))

# combine the two dictionaries and add up the values for identical keys  
all_doubl_count = {k: ref_doubl_count.get(k, 0) + alt_doubl_count.get(k, 0) for k in set(ref_doubl_count) | set(alt_doubl_count)}
print('Length of dictionary = ' + str(len(all_doubl_count))) # 3183039 

# sanity check 
## len(all_doubl_count) == (len(ref_doubl_count) + len(alt_doubl_count)) - # of keys that intersect b/n the two dictionaries  

For the next step, we get a list of samples that are in the doubleton mt and also create sample pairs out of them. We also divide the combined dictionary into two: one for when a sample is a key by itself (len(key) == 1) and the other for when the dictionary key is a pair of samples (len(key) != 1). We then go through the lists of samples obtained from the mt and see if any of them are keys in their respective doubleton dictionaries - list of samples by themselves is compared against the dictionary that has a single sample as a key and the list with sample pairs is compared against the dictionary where the key is a pair of samples. 

In [ ]:
# get list of samples from mt
mt_sample_list = mt_unrel_only2_filtered.s.collect()

# make pairs from sample list: n(n-1)/2 - 5710510 pairs 
mt_sample_pairs = [{x,y} for i, x in enumerate(sample_list) for j,y in enumerate(sample_list) if i<j]

# subset dict to only keys with length of 1 - one sample 
dict_single_samples = {x:all_doubl_count[x] for x in all_doubl_count if len(x) == 1}

    if duplicate:
        print("Removing any duplicate samples")
        # Removing any duplicates in the dataset using hl.distinct_by_col() which removes
        # columns with a duplicate column key. It keeps one column for each unique key.
        # after updating to the new dense_mt, this step is no longer necessary to run
        mt = mt.distinct_by_col()

# sanity check 
print(len(dict_single_samples) + len(dict_pair_samples) == len(all_doubl_count)) # True
# are the samples in the list the same as the dict keys?
print(len(mt_sample_list) == len(dict_single_samples)) # True 
# are the sample pairs obtained from the mt equal to what's in the pair dict? 
print(len(mt_sample_pairs) == len(dict_pair_samples)) # False - there are more sample pairs obtained from the mt

If a single sample is a key in the single-sample-key dictionary, we record the sample ID twice and it's corresponding value from the dictionary. If it is not a key, we record the sample ID twice and set the value to 0. 

In [ ]:
# single sample comparison 
single_sample_final_list = [[s, s, 0] if dict_single_samples.get(frozenset([s])) is None else [s, s, dict_single_samples[frozenset([s])]] for s in mt_sample_list]

# sanity check 
## for the single samples, the length should be consistent across dict, mt sample list, and final list
print(len(single_sample_final_list) == len(mt_sample_list) == len(dict_single_samples)) # True 

If a sample pair is a key in the sample-pair-key dictionary, we record the two sample IDs and the corresponding value from the dictionary. If that is not the case, we record the two sample IDs and set the value to 0. 

In [ ]:
# sample pair comparison
sample_pair_final_list = [[list(s)[0], list(s)[1], 0] if dict_pair_samples.get(frozenset(list(s))) is None else [list(s)[0], list(s)[1], dict_pair_samples[frozenset(list(s))]] for s in mt_sample_pairs]

# sanity check 
## length of final list should be equal to the length of the sample list obtained from the mt 
print(len(sample_pair_final_list) == len(mt_sample_pairs)) # True

Last step is to combine the two lists obtained from the comparisons, convert that into a pandas table, format it as needed, and write it out as a csv so that the values can be plotted as a heat map in R. For plot annotation purposes, we also export the sample IDs and their respective populations & genetic regions.      

In [ ]:
final_list = single_sample_final_list + sample_pair_final_list

# sanity check 
len(final_list) == len(single_sample_final_list) + len(sample_pair_final_list) # True

# format list as pandas table 
df = pd.DataFrame(final_list)
df.rename({0:'sample1', 1:'sample2', 2:'count'}, axis=1, inplace=True) # rename column names 

# save table to the Cloud so it can be plotted in R 
df.to_csv(f2_final_path, index=False, sep='\t')

# save sample IDs and their respective populations & genetic regions for heat map annotation 
sampleID_pop_reg = (mt_unrel_only2_filtered.select_cols(mt_unrel_only2_filtered['hgdp_tgp_meta']['Population'], mt_unrel_only2_filtered['hgdp_tgp_meta']['Genetic']['region'])).cols()
sampleID_pop_reg.export(f2_annotation_path, header=False)

# 3. F_st

F_st detects genetic divergence from common variance allowing us to understand past deep history. Although we are running F_st only on unrelated samples in this tutorial, unlike the F2 analysis, here we are using the filtered and pruned mt that doesn't include the 22 outliers (whole_dataset.mt = *filtered_n_pruned_output_updated.mt* - 22 outliers). 

Something to note: Since the mt we are starting with is filtered and pruned, running *hl.variant_qc* and filtering to variants with call rate > 0.05 (similar to what we did for the F2 analysis) doesn't make a difference to the number of variants.


[Back to Index](#Index)

In [ ]:
# read in the mt before running pc_relate but without outliers - 248634 variants and 4097 samples
mt_FST_initial = hl.read_matrix_table(fst_input_path) 

# filter to only unrelated samples - 3380 samples 
mt_unrel = hl.read_matrix_table(unrelated_path) 
unrel_samples = mt_unrel.s.collect() # collect sample IDs as a list 
unrel_samples = hl.literal(unrel_samples) # capture and broadcast the list as an expression 
mt_FST_unrel = mt_FST_initial.filter_cols(unrel_samples.contains(mt_FST_initial['s'])) # filter mt
print('Num of samples after filtering (unrelated samples) = ' + str(mt_FST_unrel.count()[1])) # 3380 samples

### 3a. F_st with PLINK

In order to calculate F_st using PLINK, we first need to export mt as PLINK files.

In [ ]:
# export mt as PLINK2 BED, BIM and FAM files - store on the Cloud 
hl.export_plink(mt_FST_unrel, plink_files_path, fam_id=mt_FST_unrel.hgdp_tgp_meta.Population)

The rest of the analysis is done using shell commands within the notebook. Place *"!"* before the command you want to run and proceed as if you are running codes in a terminal. You can use *"! ls"* after each run to check for ouputs in the directory and see if commands have run correctly. Also, every time you start a new cluster, you would need to download PLINK to run the F_st analysis since downloads and files are discarded when a cluster is stopped. 

Something to note: when running the notebook on the Cloud, the shell commands still run even if we didn't use "!". Not sure why but will check if that is also the case when running the notebook locally. 

3a.1. PLINK Setup 

In [ ]:
# download PLINK using a link from the PLINK website (linux - recent version - 64 bit - stable) 
! wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20210606.zip
    
# unzip the ".gz" file: 
! unzip plink_linux_x86_64_20210606.zip

# a documentation output when you run this command indicates that PLINK has been installed properly 
! ./plink 

3a.2. Files Setup 

In [ ]:
# copy the PLINK files that are stored on the Cloud into the current session directory
! gsutil cp {plink_files_path}.fam . # fam 
! gsutil cp {plink_files_path}.bim . # bim
! gsutil cp {plink_files_path}.bed . # bed

# obtain FID - in this case, it is the 78 populations in the first column of the FAM file
! awk '{print $1}' hgdp_tgp.fam | sort -u > pop.codes

# make all possible combinations of pairs using the 78 populations 
! for i in `seq 78`; do for j in `seq 78`; do if [ $i -lt $j ]; then VAR1=`sed "${i}q;d" pop.codes`; VAR2=`sed "${j}q;d" pop.codes`; echo $VAR1 $VAR2; fi; done; done > pop.combos

# sanity check 
! wc -l pop.combos # 3003

# create directories for intermediate files and F_Sst results 
! mkdir within_files # intermediate files
! mkdir FST_results # F_st results

3a.3. Scripts Setup 

In [9]:
### script 1 ####

# for each population pair, set up a bash script to create a "within" file and run F_st 
# files to be produced: 
### [pop_pairs].within will be saved in the "within_files" directory
### [pop_pairs].fst, [pop_pairs].log, and [pop_pairs].nosex will be save in "FST_results" directory

fst_script = '''    
#!/bin/bash

# set variables
for i in `seq 3003`
do
    POP1=`sed "${i}q;d" pop.combos | awk '{ print $1 }'`
    POP2=`sed "${i}q;d" pop.combos | awk '{ print $2 }'`

# create "within" files for each population pair using the FAM file (columns 1,2 and 1 again)
    awk -v r1=$POP1 -v r2=$POP2 '$1 == r1 || $1 == r2' hgdp_tgp.fam | awk '{ print $1, $2, $1 }' > within_files/${POP1}_${POP2}.within

# run F_st
    ./plink --bfile hgdp_tgp --fst --within within_files/${POP1}_${POP2}.within --out FST_results/${POP1}_${POP2}
done'''

with open('run_fst.py', mode='w') as file:
    file.write(fst_script)

In [ ]:
### script 2 ### - script 1 has to be run for this script to run 

# use the "[pop_pairs].log" file produced from script 1 above (located in "FST_results" directory) to get the "Mean F_st estimate" for each population pair and compile all of the values in a single file for F_st heat map generation

extract_mean_script = ''' 
#!/bin/bash

# set variables
for i in `seq 3003`
do
    POP1=`sed "${i}q;d" pop.combos | awk '{ print $1 }'`
    POP2=`sed "${i}q;d" pop.combos | awk '{ print $2 }'`
    mean_FST=$(tail -n4 FST_results/${POP1}_${POP2}.log | head -n 1 | awk -F: '{print $2}' | awk '{$1=$1};1')
    printf "%-20s\t%-20s\t%-20s\n " ${POP1} ${POP2} $mean_FST >> mean_fst_sum.txt
done'''

with open('extract_mean.py', mode='w') as file:
    file.write(extract_mean_script)

3a.4. Run Scripts 

In [ ]:
# run script 1 and direct the run log into a file (~20min to run) 
! sh run_fst.py > fst_script.log

# sanity check 
! cd within_files/; ls | wc -l # 3003
! cd FST_results/; ls | wc -l # 3003 * 3 = 9009 

In [ ]:
# run script 2; requires script 1 to be run first (~ 1min to run)
! sh extract_mean.py 

# copy script 2 output to the Cloud for heat map plotting in R 
! gsutil cp mean_fst_sum.txt {fst_final_path}